Wstępny kod z funkcjami, ktorych nie mozemy uzywac, zeby zobaczyc czy wyjdzie

In [4]:
import cv2

# Ładowanie klasyfikatora Haar Cascade dla detekcji ludzi
person_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_fullbody.xml')

# Funkcja do usuwania ludzi z obrazu
def remove_people(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Detekcja ludzi na obrazie
    people = person_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Usunięcie ludzi z obrazu
    for (x, y, w, h) in people:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 0), -1) # Rysowanie prostokąta, który zakryje obszar z ludźmi
    
    return image

# Ładowanie wszystkich 5 obrazów i usuwanie ludzi
images = []
for i in range(1, 6):
    image_path = f"cathedral/d00{i}.jpg"
    img = cv2.imread(image_path)
    img = remove_people(img)
    images.append(img)

# Obliczanie współrzędnych punktów, w których obrazy zostaną nakładane na siebie
height, width, _ = images[0].shape
offset_x = width // 2
offset_y = height // 2

# Nakładanie obrazów na siebie
combined_image = images[0]
for i in range(1, len(images)):
    combined_image = cv2.addWeighted(combined_image, 0.5, images[i], 0.5, 0)

# Zapisanie obrazu bez ludzi
cv2.imwrite("combined_no_people_overlaid.jpg", combined_image)


True